# HW2

In [2]:
# Common imports
import numpy as np
import pandas as pd


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
home = pd.read_csv("home_data.csv")
home.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## PART I

### 1. Add Extra Features 
Add 4 new variables in the data set.   
* ‘bedrooms_squared’ = ‘bedrooms’*‘bedrooms’ 
* ‘bed_bath_rooms’ = ‘bedrooms’*‘bathrooms’ 
* ‘log_sqft_living’ = log(‘sqft_living’) 
* ‘lat_plus_long’ = ‘lat’ + ‘long’ 


Before we continue let’s explain these new variables: 
* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and 
lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly 
affect houses with many bedrooms. 
* Bedrooms times bathrooms is what's called an "interaction" variable. It is large when both 
of them are large. 
* Taking the log of square feet has the effect of bringing large values closer together and 
spreading out small values. 
* Adding latitude to longitude is non-sensical but we will do it anyway (you'll see why) 
 
What are the mean (arithmetic average) values of your 4 new variables? (round to 2 digits) 

In [4]:
home["bedrooms_squared"] = list(map(lambda x: x**2, home["bedrooms"]))
home["bed_bath_rooms"] = list(map(lambda x, y: x * y, home["bedrooms"], home['bathrooms']))
home["log_sqft_living"] = list(map(lambda x: np.log(x), home["sqft_living"]))
home["lat_plus_long"] = list(map(lambda x, y: x + y, home["lat"], home['long']))

In [11]:
print("the mean (arithmetic average) values of")
home[["bedrooms_squared","bed_bath_rooms","log_sqft_living","lat_plus_long"]].mean().round(2)

the mean (arithmetic average) values of


bedrooms_squared    12.23
bed_bath_rooms       7.50
log_sqft_living      7.55
lat_plus_long      -74.65
dtype: float64

: 

In [5]:
home.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bedrooms_squared,bed_bath_rooms,log_sqft_living,lat_plus_long
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1.0,0,0,...,0,98178,47.5112,-122.257,1340,5650,9,3.00,7.073270,-74.7458
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2.0,0,0,...,1991,98125,47.7210,-122.319,1690,7639,9,6.75,7.851661,-74.5980
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1.0,0,0,...,0,98028,47.7379,-122.233,2720,8062,4,2.00,6.646391,-74.4951
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1.0,0,0,...,0,98136,47.5208,-122.393,1360,5000,16,12.00,7.580700,-74.8722
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1.0,0,0,...,0,98074,47.6168,-122.045,1800,7503,9,6.00,7.426549,-74.4282


### 2. Train-Test Split 
Split the data into training set (80%) and test set (20%) with random state = 0.  

In [6]:
from sklearn.model_selection import train_test_split

X = home.drop("price", axis=1)
y = home["price"]
X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 0)

(21613, 24)

In [7]:
X_train.shape
X_test.shape

(17290, 24)

(4323, 24)

### 3. Estimate the regression coefficients/weights for predicting ‘price’ for the following three models. 
* Model 1: ‘sqft_living’, ‘bedrooms’, ‘bathrooms’, ‘lat’, and ‘long’ 
* Model 2: ‘sqft_living’, ‘bedrooms’, ‘bathrooms’, ‘lat’,‘long’, and ‘bed_bath_rooms’ 
* Model 3: ‘sqft_living’, ‘bedrooms’, ‘bathrooms’, ‘lat’,‘long’, ‘bed_bath_rooms’, 
‘bedrooms_squared’, ‘log_sqft_living’, and ‘lat_plus_long’ 

You’ll note that the three models here are “nested” in that all of the features of the Model 1 are in 
Model 2 and all of the features of Model 2 are in Model 3.  


In [8]:
M1 = ["sqft_living", "bedrooms", "bathrooms", "lat", "long"]
M2 = ["sqft_living", "bedrooms", "bathrooms", "lat", "long", "bed_bath_rooms"]
M3 = ["sqft_living", "bedrooms", "bathrooms", "lat", "long", "bed_bath_rooms", "bedrooms_squared", "log_sqft_living", "lat_plus_long" ]

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# build the models with pipline 
pipline1 = make_pipeline(LinearRegression() )
pipline2 = make_pipeline(LinearRegression() )
pipline3 = make_pipeline(LinearRegression() )

linReg1 = pipline1.fit( X_train[M1] , y_train ).named_steps['linearregression']
linReg2 = pipline2.fit( X_train[M2] , y_train ).named_steps['linearregression']
linReg3 = pipline3.fit( X_train[M3] , y_train ).named_steps['linearregression']

# Get coefficients with feature names
pd.DataFrame({'Feature': X[M1].columns, 'Coefficient': linReg1.coef_ }).round({'Coefficient': 3})
pd.DataFrame({'Feature': X[M2].columns, 'Coefficient': linReg2.coef_ }).round({'Coefficient': 3})
pd.DataFrame({'Feature': X[M3].columns, 'Coefficient': linReg3.coef_ }).round({'Coefficient': 3})

# Get intercept with feature names
print("The intercept of all 3 Models")
print("The intercept of model 1: ", linReg1.intercept_)
print("The intercept of model 2: ", linReg2.intercept_)
print("The intercept of model 3: ", linReg3.intercept_)

#linReg1.score(X_train[M1] , y_train)
#linReg2.score(X_train[M2] , y_train)
#linReg3.score(X_train[M3] , y_train)

,Feature,Coefficient
0,sqft_living,312.942
1,bedrooms,-53096.268
2,bathrooms,14777.042
3,lat,653983.345
4,long,-325707.345


,Feature,Coefficient
0,sqft_living,306.820
1,bedrooms,-104604.713
2,bathrooms,-70181.522
3,lat,650590.954
4,long,-309965.761
5,bed_bath_rooms,24944.148


,Feature,Coefficient
0,sqft_living,537.808
1,bedrooms,2780.485
2,bathrooms,101363.772
3,lat,530798.411
4,long,-409655.443
5,bed_bath_rooms,-18182.257
6,bedrooms_squared,724.580
7,log_sqft_living,-571030.023
8,lat_plus_long,121142.968


The intercept of all 3 Models
The intercept of model 1:  -70870847.44367687
The intercept of model 2:  -68606821.79348008
The intercept of model 3:  -62628451.67823651


What is the sign (positive or negative) for the coefficient/weight for ‘bathrooms’ in Model 1? 

What is the sign (positive or negative) for the coefficient/weight for ‘bathrooms’ in Model 2? 

Is the sign for the coefficient the same in both models? Think about why this might be the case.

In Model 1, the sign for the coefficient/weight for 'bathrooms' is positive (14777.042), and in Model 2, 
the sign for the coefficient/weight for 'bathrooms' is also positive (-70181.522).


The change in the sign of the 'bathrooms' coefficient between Model 1 and Model 2 can be attributed to the introduction of the interaction term 'bed_bath_rooms'. 

In Model 1:

'bathrooms' has a positive coefficient (14777.042), suggesting that an increase in the number of bathrooms is associated with a higher predicted target value.

In Model 2:

The introduction of the interaction term 'bed_bath_rooms' captures the joint effect of 'bedrooms' and 'bathrooms'. The coefficient for 'bed_bath_rooms' is positive (106298.615), suggesting that, when considering the interaction, an increase in the combined effect of 'bedrooms' and 'bathrooms' is associated with a higher predicted target value.
However, the individual coefficient for 'bathrooms' becomes negative (-70181.522) in Model 2.

This shift in the sign of the 'bathrooms' coefficient is likely due to the correlation or interaction between 'bedrooms' and 'bathrooms'. When an interaction term is introduced, it can capture joint effects that may not be apparent when considering the individual features in isolation. In this case, the interaction term may be highlighting a relationship where the combined effect of 'bedrooms' and 'bathrooms' has a different impact on the target variable compared to considering 'bathrooms' alone.

### 4. Now using your three estimated models compute the RSS (Residual Sum of Squares) on the Training data. Which model (1, 2 or 3) had the lowest RSS on TRAINING data?

In [63]:
pred1_train = linReg1.predict(X_train[M1])
pred2_train = linReg2.predict(X_train[M2])
pred3_train = linReg3.predict(X_train[M3])

lin1_train_RSS = sum((y_train- pred1_train)**2)
lin2_train_RSS = sum((y_train- pred2_train)**2)
lin3_train_RSS = sum((y_train- pred3_train)**2)

# Assuming lin1_RSS, lin2_RSS, lin3_RSS are the RSS values for M1, M2, and M3
rss_values = [lin1_train_RSS, lin2_train_RSS, lin3_train_RSS]

# Creating the DataFrame and adding the 'Rank' column
pd.DataFrame({'Model': ["M1", "M2", "M3"], 'RSS': rss_values}).assign(Rank=lambda x: x['RSS'].rank())

,Model,RSS,Rank
0,M1,9.798436e+14,3.0
1,M2,9.707992e+14,2.0
2,M3,9.136536e+14,1.0


Model 3 has the lowest RSS on Training set

### 5. Now using your three estimated models compute the RSS on the Testing data. Which model (1, 2, or 3) had the lowest RSS on TESTING data? 

In [65]:
pred1_test = linReg1.predict(X_test[M1])
pred2_test = linReg2.predict(X_test[M2])
pred3_test = linReg3.predict(X_test[M3])

lin1_test_RSS = sum((y_test - pred1_test)**2)
lin2_test_RSS = sum((y_test - pred2_test)**2)
lin3_test_RSS = sum((y_test - pred3_test)**2)

# Assuming lin1_RSS, lin2_RSS, lin3_RSS are the RSS values for M1, M2, and M3
rss_values2 = [lin1_test_RSS, lin2_test_RSS, lin3_test_RSS]

# Creating the DataFrame and adding the 'Rank' column
pd.DataFrame({'Model': ["M1", "M2", "M3"], 'RSS': rss_values2}).assign(Rank=lambda x: x['RSS'].rank())

,Model,RSS,Rank
0,M1,2.134871e+14,3.0
1,M2,2.107785e+14,2.0
2,M3,2.039721e+14,1.0


Model 3 has the lowest RSS on testing set

In [70]:
# Addition if want to do standard scaler

# Build pipelines
pipeline1 = make_pipeline(StandardScaler(), LinearRegression())
pipeline2 = make_pipeline(StandardScaler(), LinearRegression())
pipeline3 = make_pipeline(StandardScaler(), LinearRegression())

# Fit the pipelines and get the linear regression models
linReg1 = pipeline1.fit(X_train[M1], y_train).named_steps['linearregression']
linReg2 = pipeline2.fit(X_train[M2], y_train).named_steps['linearregression']
linReg3 = pipeline3.fit(X_train[M3], y_train).named_steps['linearregression']

# Transform the training set with the fitted StandardScaler
X_train_scaled_M1 = pipeline1.named_steps['standardscaler'].transform(X_train[M1])
X_train_scaled_M2 = pipeline2.named_steps['standardscaler'].transform(X_train[M2])
X_train_scaled_M3 = pipeline3.named_steps['standardscaler'].transform(X_train[M3])

# Transform the training set with the fitted StandardScaler
X_test_scaled_M1 = pipeline1.named_steps['standardscaler'].transform(X_test[M1])
X_test_scaled_M2 = pipeline2.named_steps['standardscaler'].transform(X_test[M2])
X_test_scaled_M3 = pipeline3.named_steps['standardscaler'].transform(X_test[M3])

# Predictions on the scaled training set
pred1_train = linReg1.predict(X_train_scaled_M1)
pred2_train = linReg2.predict(X_train_scaled_M2)
pred3_train = linReg3.predict(X_train_scaled_M3)

# Calculate RSS for each model
lin1_train_RSS = sum((y_train - pred1_train) ** 2)
lin2_train_RSS = sum((y_train - pred2_train) ** 2)
lin3_train_RSS = sum((y_train - pred3_train) ** 2)

# Assuming lin1_RSS, lin2_RSS, lin3_RSS are the RSS values for M1, M2, and M3
rss_values = [lin1_train_RSS, lin2_train_RSS, lin3_train_RSS]

# Creating the DataFrame and adding the 'Rank' column
pd.DataFrame({'Model': ["M1", "M2", "M3"], 'RSS': rss_values}).assign(Rank=lambda x: x['RSS'].rank())

# Predictions on the scaled testing set
pred1_test = linReg1.predict(X_test_scaled_M1)
pred2_test = linReg2.predict(X_test_scaled_M2)
pred3_test = linReg3.predict(X_test_scaled_M3)

# Calculate RSS for each model
lin1_test_RSS = sum((y_test - pred1_test) ** 2)
lin2_test_RSS = sum((y_test - pred2_test) ** 2)
lin3_test_RSS = sum((y_test - pred3_test) ** 2)

# Assuming lin1_RSS, lin2_RSS, lin3_RSS are the RSS values for M1, M2, and M3
rss_values2 = [lin1_test_RSS, lin2_test_RSS, lin3_test_RSS]

# Creating the DataFrame and adding the 'Rank' column
pd.DataFrame({'Model': ["M1", "M2", "M3"], 'RSS': rss_values2}).assign(Rank=lambda x: x['RSS'].rank())

,Model,RSS,Rank
0,M1,9.798436e+14,3.0
1,M2,9.707992e+14,2.0
2,M3,9.136520e+14,1.0


,Model,RSS,Rank
0,M1,2.134871e+14,3.0
1,M2,2.107785e+14,2.0
2,M3,2.039748e+14,1.0


### 6. Train-Validate-Test 

Split the original data into train-validate set (90%) and test set (10%) using random state = 0.  

Use GridSearchCV and 5-fold cross validation to compare the performance of the following 

polynomial regression models on the train-validation set. 

Build the following polynomial regression models to predict house prices using just ‘sqft_living’  

Polynomial degree in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]  

Which model is the best? Report best model’s performance on the test set.

In [48]:
X = home[["sqft_living"]]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=0)

In [49]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV

pipe = make_pipeline(PolynomialFeatures(), StandardScaler(),LinearRegression())
param_grid = {'polynomialfeatures__degree': range(1,16)}

grid = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1, return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('standardscaler', StandardScaler()),
                                       ('linearregression',
                                        LinearRegression())]),
             n_jobs=-1, param_grid={'polynomialfeatures__degree': range(1, 16)},
             return_train_score=True)

In [50]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_polynomialfeatures__degree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.009602,0.000490,0.002601,0.000800,1,{'polynomialfeatures__degree': 1},0.490820,0.514283,4.761793e-01,0.500995,...,4.926476e-01,1.377603e-02,6,0.495592,0.488714,0.498497,0.492889,0.497444,0.494627,0.003517
1,0.011409,0.001491,0.003009,0.000647,2,{'polynomialfeatures__degree': 2},0.541289,0.587065,4.249993e-01,0.542441,...,5.234550e-01,5.370697e-02,1,0.532704,0.518645,0.554639,0.532445,0.537395,0.535166,0.011581
2,0.016708,0.007871,0.003005,0.000623,3,{'polynomialfeatures__degree': 3},0.537095,0.568769,4.197636e-01,0.545823,...,5.189834e-01,5.175265e-02,3,0.542252,0.531113,0.554651,0.540130,0.545135,0.542656,0.007613
3,0.013305,0.004996,0.003401,0.000799,4,{'polynomialfeatures__degree': 4},0.539351,0.571139,3.287662e-01,0.543402,...,5.009453e-01,8.751742e-02,5,0.545257,0.534565,0.555736,0.544136,0.548741,0.545687,0.006879
4,0.012203,0.000980,0.002600,0.000490,5,{'polynomialfeatures__degree': 5},0.547411,0.585608,3.299410e-01,0.541550,...,5.041779e-01,8.989329e-02,4,0.548409,0.536283,0.555736,0.550011,0.554915,0.549071,0.006977
5,0.020606,0.003722,0.003600,0.000491,6,{'polynomialfeatures__degree': 6},0.553602,0.603056,3.919472e-01,0.544669,...,5.234226e-01,7.069594e-02,2,0.554605,0.540025,0.561013,0.557253,0.561094,0.554798,0.007779
6,0.023607,0.003773,0.003000,0.000004,7,{'polynomialfeatures__degree': 7},0.554194,0.598816,-1.503953e+00,0.543645,...,1.429287e-01,8.238221e-01,8,0.554645,0.540050,0.562205,0.557684,0.561603,0.555237,0.008076
7,0.029610,0.006055,0.004398,0.001496,8,{'polynomialfeatures__degree': 8},0.556740,0.596102,-2.978667e+00,0.541797,...,-1.525162e-01,1.413287e+00,9,0.554988,0.540198,0.562224,0.559123,0.562670,0.555841,0.008290
8,0.036410,0.008164,0.004801,0.000751,9,{'polynomialfeatures__degree': 9},0.556635,0.606019,8.766597e-02,0.541740,...,4.627029e-01,1.895859e-01,7,0.554994,0.540209,0.562239,0.559126,0.562671,0.555848,0.008288
9,0.060415,0.025293,0.009000,0.005178,10,{'polynomialfeatures__degree': 10},0.556776,-5.915815,-2.958826e+01,0.540511,...,-6.777078e+00,1.167641e+01,10,0.555003,0.540897,0.562268,0.559376,0.562726,0.556054,0.008062


In [51]:
# Get the best estimator from the grid search
best_poly_model = grid.best_estimator_

print(f"The best model’s performance on the test set is: { best_poly_model.named_steps["polynomialfeatures"].degree}")
print(f"The best model’s performance on the test set is: { best_poly_model.score(X_test, y_test)}")

The best model’s performance on the test set is: 2
The best model’s performance on the test set is: 0.5115228302233856


## PART II

### 1. Load the breast cancer dataset. Import the functions `load_breast_cancer` from `sklearn.datasets`.  

In [159]:
from sklearn.datasets import load_breast_cancer
b_cancer = load_breast_cancer()
X

### 2. Split the data into train-validate set (90%) and test set (10%) using random_state = 0.

In [160]:
X_trainVal, X_test, y_trainVal, y_test = train_test_split(
    b_cancer['data'], b_cancer['target'], random_state=0, test_size=0.1)

### 3. Split the train-validate set into training set (80%) and validation set (20%) using  random_state = 0. Build KNN models with n_neighbors = {1, 3, 5, 7, 9} using the training set. Compare their performance on the validation set and pick the best model. Report the best model’s performance on the test set. 

In [208]:
from sklearn.neighbors import KNeighborsClassifier

X_train, X_val, y_train, y_val = train_test_split(
    X_trainVal, y_trainVal, test_size=0.2, random_state=0)

val_scores = [] 
test_scores = []
neighbors = np.arange(1, 10, 2)

for n_neighbors in range(1,10,2):
    knn = KNeighborsClassifier(n_neighbors)
    knn.fit(X_train, y_train)
    print("n_neighbors is eqaul to : {}".format(n_neighbors))
    print("Training set score: {:.4f}".format(knn.score(X_train, y_train)))
    print("Validation set score: {:.4f}".format(knn.score(X_val, y_val)))
    print("Test set score: {:.4f}".format(knn.score(X_test, y_test)))
    print("\n")
    val_scores.append(knn.score(X_val, y_val))
    test_scores.append(knn.score(X_test, y_test))

print(f"The best validation score: {np.max(val_scores):.3}")
print("The best n_neighbors:", neighbors[np.argmax(val_scores)])
print(f"The test score on the best n_neighbors model: {np.max(test_scores):.3}")

KNeighborsClassifier(n_neighbors=1)

n_neighbors is eqaul to : 1
Training set score: 1.0000
Validation set score: 0.8932
Test set score: 0.8772




KNeighborsClassifier(n_neighbors=3)

n_neighbors is eqaul to : 3
Training set score: 0.9584
Validation set score: 0.9320
Test set score: 0.8947




KNeighborsClassifier()

n_neighbors is eqaul to : 5
Training set score: 0.9535
Validation set score: 0.9417
Test set score: 0.9298




KNeighborsClassifier(n_neighbors=7)

n_neighbors is eqaul to : 7
Training set score: 0.9389
Validation set score: 0.9515
Test set score: 0.9474




KNeighborsClassifier(n_neighbors=9)

n_neighbors is eqaul to : 9
Training set score: 0.9413
Validation set score: 0.9515
Test set score: 0.9474


The best validation score: 0.951
The best n_neighbors: 7
The test score on the best n_neighbors model: 0.947


### 4. Use GridSearchCV and 5-fold cross validation to build and compare KNN models with n_neighbors = {1, 3, 5, 7, 9, 11, 13, 15, 17, 19} using the train-validate set. Report the best model’s performance on the test set.  

In [173]:
param_grid = {'n_neighbors':  np.arange(1, 20, 2) }

grid = GridSearchCV(KNeighborsClassifier(),param_grid, cv=5, return_train_score=True)
grid.fit(X_trainVal,y_trainVal)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19])},
             return_train_score=True)

In [206]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.0002,0.00040,0.006802,7.484306e-04,1,{'n_neighbors': 1},0.941748,0.951456,0.882353,0.931373,...,0.925700,0.023871,10,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
1,0.0004,0.00049,0.006802,1.721017e-03,3,{'n_neighbors': 3},0.932039,0.922330,0.931373,0.911765,...,0.931658,0.016318,5,0.953545,0.960880,0.970732,0.953659,0.965854,0.960934,0.006749
2,0.0004,0.00049,0.005601,7.997039e-04,5,{'n_neighbors': 5},0.970874,0.922330,0.901961,0.911765,...,0.931582,0.025596,8,0.941320,0.948655,0.953659,0.956098,0.941463,0.948239,0.006084
3,0.0004,0.00049,0.005001,4.862804e-07,7,{'n_neighbors': 7},0.970874,0.941748,0.911765,0.921569,...,0.937426,0.020305,2,0.938875,0.943765,0.951220,0.953659,0.939024,0.945309,0.006131
4,0.0006,0.00049,0.005401,4.898433e-04,9,{'n_neighbors': 9},0.970874,0.932039,0.901961,0.921569,...,0.937445,0.025309,1,0.933985,0.938875,0.948780,0.946341,0.939024,0.941401,0.005402
5,0.0002,0.00040,0.004601,4.900961e-04,11,{'n_neighbors': 11},0.970874,0.912621,0.911765,0.911765,...,0.931601,0.024759,6,0.936430,0.941320,0.948780,0.943902,0.934146,0.940916,0.005232
6,0.0008,0.00040,0.004801,3.997088e-04,13,{'n_neighbors': 13},0.970874,0.912621,0.911765,0.921569,...,0.933562,0.023465,3,0.931540,0.936430,0.941463,0.941463,0.934146,0.937009,0.003953
7,0.0004,0.00049,0.004801,4.001619e-04,15,{'n_neighbors': 15},0.970874,0.912621,0.921569,0.921569,...,0.933562,0.020863,3,0.933985,0.938875,0.934146,0.936585,0.934146,0.935548,0.001924
8,0.0000,0.00000,0.005601,4.898041e-04,17,{'n_neighbors': 17},0.970874,0.912621,0.901961,0.931373,...,0.931601,0.023970,6,0.931540,0.933985,0.931707,0.929268,0.931707,0.931642,0.001493
9,0.0002,0.00040,0.005601,4.900182e-04,19,{'n_neighbors': 19},0.970874,0.912621,0.892157,0.921569,...,0.927679,0.026735,9,0.929095,0.933985,0.931707,0.929268,0.931707,0.931153,0.001812


In [213]:
best_knn_model = grid.best_estimator_
print("The best n_neighbors:", grid.best_params_['n_neighbors'])
print("The validation score on the best n_neighbors model:",np.max(grid.cv_results_["mean_test_score"]))
print(f"The test score on the best n_neighbors model: {best_knn_model.score(X_test, y_test):.3}")

The best n_neighbors: 9
The validation score on the best n_neighbors model: 0.9374452693698838
The test score on the best n_neighbors model: 0.947
